In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

train_data_dir = r'Train_Dataset'
class_names = sorted(os.listdir(train_data_dir))
num_classes = len(class_names)

img_width, img_height = 256, 256
batch_size = 40

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    #class_mode='binary',
    shuffle=True
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    #Conv2D(128, (3, 3), activation='relu'),
    #MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
    #Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    #loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=15
)

In [ ]:
import cv2
import serial
import time
import pymysql
import keras_ocr
import numpy as np
import tensorflow as tf
#database

img_width, img_height = 256, 256

host = 'localhost'
user = 'root'
password = ''
database = 'project'

connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

#serial

device = 'COM6'
print("Trying...", device)
arduino = serial.Serial(device, 9600,timeout=1)


def classify_single_image(image_path, threshold=0.5):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predicted_probabilities = model.predict(img_array)[0]
    
    predicted_class_index = np.argmax(predicted_probabilities)
    predicted_class_probability = predicted_probabilities[predicted_class_index]
    
    if predicted_class_probability > threshold:
        predicted_class = class_names[predicted_class_index]
    else:
        predicted_class = 'unknown'
    
    return predicted_class




def detectColor(image):
    color_names = {
        "red": (255, 0, 0),
        "green": (0, 255, 0),
        "blue": (0, 0, 255),
    }
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixels = image.reshape(-1, 3)
    mean_color = np.mean(pixels, axis=0)
    rgb_color = tuple(mean_color.astype(int))
    
    min_distance = float('inf')
    closest_color = None
    for color_name, color_value in color_names.items():
        distance = np.linalg.norm(np.array(rgb_color) - np.array(color_value))
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name

    return closest_color




def detectText(image):
    pipeline = keras_ocr.pipeline.Pipeline()
    images = [keras_ocr.tools.read(url) for url in [image]]
    
    prediction_groups = pipeline.recognize(images)
    predicted_image_1 = prediction_groups[0]
    final_text=""
    for text, box in predicted_image_1:
        final_text+=text
    return final_text




def detectObject(founder):
    cam = cv2.VideoCapture(1)
    result, image = cam.read()
        
    cv2.imwrite('capture.jpg',image)

    image_path = r'capture.jpg'
    
    predicted_class = classify_single_image(image_path)
    color = detectColor(image)
    text = detectText(image)
    
    category = 0
    
    if(predicted_class == 'handbag1' or predicted_class == 'handbag2'):
        category = 2
    elif(predicted_class == 'pendrive1' or predicted_class == 'pendrive2'):
        category = 1
    else:
        category = 3
        
    print(f"Predicted class: {predicted_class}")
    print("detected color : ",color)
    print("detected text : ",text)
    
    cursor = connection.cursor()
    update_query = "UPDATE items SET color = %s, text = %s, found = %s WHERE id = %s"
    cursor.execute(update_query, (color, text, founder, category))
    connection.commit()
    cursor.close()
    
    return category


def receiveItem(r_color, r_text, r_person, r_category):
    cursor = connection.cursor()
    select_query = "SELECT * FROM items WHERE color = %s AND text = %s AND id = %s"
    cursor.execute(select_query, (r_color,r_text,r_category))
    results = cursor.fetchall()
    print("inside : ",r_color,r_text,r_category)
    
    if(results):
        query = "UPDATE items SET color = %s, text = %s, lost = %s WHERE id = %s"
        cursor.execute(query, ("none", "none", r_person, r_category))
        connection.commit()
        cursor.close()
        return "ok"
    else:
        return "notok"
    
    
    
    
    
while True:
    #time.sleep(1)
    
    while(arduino.inWaiting()==0):
        pass 
    receive = arduino.readline().decode().strip()
    #print(receive)
     
    signal = "0"
    category = "0"
    take_color = "none"
    take_text = "none"
    take_person = "none"
    pieces = receive.split('#')
    if len(pieces) == 5:
        signal = pieces[0]
        category = pieces[1]
        take_color = pieces[2]
        take_text = pieces[3]
        take_person = pieces[4]

    
    
    if(signal == "1"):
        print("detecting item...")
        ok = detectObject(take_person)
        send_category = str(ok)
        arduino.write(send_category.encode())
        time.sleep(1)
        continue
    elif(signal == "2"):
        print("matching item...")
        #print(take_color,take_text,category)
        send_feedback = receiveItem(take_color,take_text,take_person,category)
        arduino.write(send_feedback.encode())
        time.sleep(1)
        continue
        
    #print("go...")
        
arduino.close()
connection.close()